<a href="https://colab.research.google.com/github/1ser1n1/Object-Oriented-Programming/blob/main/2_%EA%B0%9C%EB%B0%A9_%ED%8F%90%EC%87%84_%EC%9B%90%EC%B9%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. 개방 폐쇄 원칙(Open/closed principle)**




> **클래스는 확장에 열려 있어야하며, 수정에는 닫혀 있어야 한다.**

확장에 열려 있다는 것은 프로그램의 기존 기능을 확장할 수 있다는 것이고, 수정에 닫혀 있다는 것은 한 번 작성한 코드를 바꾸지 않아도 된다는 것이다.

=> *어떤 클래스의 코드를 수정하지 않아도 기존 기능을 확장할 수 있어야 된다.*

개방폐쇄 원칙을 지키는 이유: 더 쉽게 협력하고 더 편하게 수정하기 위해서!!



In [1]:
class MessageNotificationManager:
    """메시지 알림 관리 클래스"""
    def __init__(self):
        self.message_notifications = []

    def add_new_message(self, new_message):
        """새로 온 메시지 추가"""
        self.message_notifications.append(new_message)

    def display_message_notifications(self):
        """모든 새 메시지 확인"""
        print("새로 온 메시지들:")

        for message in self.message_notifications:
            print(message.short_message() + "\n")


class KakaoTalkMessage:
    """카카오톡 메시지 클래스"""
    notification_message_max_len = 10

    def __init__(self, sent_by, time, content):
        self.sent_by = sent_by
        self.time = time
        self.content = content

    def short_message(self):
        """메시지의 정보와 내용을 리턴함"""
        message_str = "{}\n{}\n".format(self.time, self.sent_by)
        message_str += self.content if len(self.content) <= KakaoTalkMessage.notification_message_max_len else self.content[:KakaoTalkMessage.notification_message_max_len] + "..."

        return message_str


class FacebookMessage:
    """페이스북 메시지 클래스"""
    notification_message_max_len = 15

    def __init__(self, sent_by, location, time, content):
        self.sent_by = sent_by
        self.location = location
        self.time = time
        self.content = content

    def notification_display_info(self):
        """메시지를 짧은 형태로 리턴함"""
        res_str = "{}\n{}\n{}\n".format(self.time, self.sent_by, self.location)
        res_str += self.content if len(self.content) <= FacebookMessage.notification_message_max_len else self.content[:FacebookMessage.notification_message_max_len] + "..."

        return res_str   
        

class TextMessage:
    """문자 메시지 클래스"""
    notification_message_max_len = 12

    def __init__(self, sent_by, time, content):
        self.sent_by = sent_by
        self.time = time
        self.content = content

    def notification_string(self):
        """메시지의 정보와 내용을 리턴함"""
        noti_string = "{}, {}\n".format(self.sent_by, self.time)
        noti_string += self.content if len(self.content) <= TextMessage.notification_message_max_len else self.content[:TextMessage.notification_message_max_len] + "..."
        return noti_string 



# 메시지 알림 관리 인스턴스 생성
message_notification_manager = MessageNotificationManager()

# 서로 다른 종류의 메시지 3개 생성
kakao_talk_message = KakaoTalkMessage("고대위", "2019년 7월 1일 오후 11시 30분", "나 오늘 놀러 못갈 거 같아, 미안!")
facebook_message = FacebookMessage("고대위", "서울시 성북구", "2019년 7월 1일 오후 11시 35분", "아니다, 갈게! 너네 어디서 놀고 있어?")
text_message = TextMessage("이영훈", "2019년 7월 2일 오전 12시 30분", "나도 놀러 갈게, 나 지금 출발")

# 메시지 알림 관리 인스턴스에 3개의 메시지를 추가
message_notification_manager.add_new_message(kakao_talk_message)
message_notification_manager.add_new_message(facebook_message)
message_notification_manager.add_new_message(text_message)

# 메시지 알림 관리 인스턴스에 있는 모든 메시지 출력
message_notification_manager.display_message_notifications()            


새로 온 메시지들:
2019년 7월 1일 오후 11시 30분
고대위
나 오늘 놀러 못갈...



AttributeError: ignored

위의 MessageNotificationManager 클래스는 KakaoTalkMessage 클래스를 사용하는 것만 생각하고 작성했던 클래스이다. 
그래서 만약 새로 만든 MessageNotificationManager 클래스의 display_message_notifications 메소드를 호출할 때 아래 코드 부분의

```
# for message in self.message_notifications:
            print(message.short_message() + "\n")
```
short_message 메소드 부분에서 에러가 날 것이다. 왜냐면 FacebookMessage 클래스와 TextMessage 클래스에는 short_message라는 메소드가 없으니까이다.

그래서 개방 폐쇄 원칙에 어긋나지 않게 MessageNotificationManager 클래스의 코드를 더 이상 수정하지 않아도 확장이 용이할 수 있는 상태로 만들어 보자!!


## **수정본**



---
**알림 가능한 메시지(NotifiableMessage) 추상 클래스**


```
# from abc import ABC, abstractmethod

class NotifiableMessage(ABC):
    """알림 가능한 메시지를 나타내는 추상 클래스"""
    @abstractmethod
    def get_notification_message(self) -> str:
        """알림 내용에 들어갈 문자열을 리턴하는 메소드"""
        pass
```
이제 모든 종류의 메시지 클래스들이 이 NotifiableMessage 추상 클래스를 상속받게 하면 된다.


**카카오톡(KakaoTalkMessage) 클래스**


```
class KakaoTalkMessage(NotifiableMessage):
    """카카오톡 메시지 클래스"""
    notification_message_max_len = 10

    def __init__(self, sent_by, time, content):
        self.sent_by = sent_by
        self.time = time
        self.content = content

    def get_notification_message(self):
        """메시지의 정보와 내용을 리턴함"""
        message_str = "{}\n{}\n".format(self.time, self.sent_by)
        message_str += self.content if len(self.content) <= KakaoTalkMessage.notification_message_max_len else self.content[:KakaoTalkMessage.notification_message_max_len] + "..."

        return message_str
```

**페이스북 메시지(FacebookMessage) 클래스**



```
class FacebookMessage(NotifiableMessage):
    """페이스북 메시지 클래스"""
    notification_message_max_len = 15

    def __init__(self, sent_by, location, time, content):
        self.sent_by = sent_by
        self.location = location
        self.time = time
        self.content = content

    def get_notification_message(self):
        """메시지의 정보와 내용을 리턴함"""
        res_str = "{}\n{}\n{}\n".format(self.time, self.sent_by, self.location)
        res_str += self.content if len(self.content) <= FacebookMessage.notification_message_max_len else self.content[:FacebookMessage.notification_message_max_len] + "..."

        return res_str

```
**메시지 알림 매니저(MessageNotificationManager) 클래스**

```
class MessageNotificationManager:
    """메시지 알림 관리 클래스"""
    def __init__(self):
        self.message_notifications = []

    def add_new_message(self, new_message: NotifiableMessage):
        """새로 온 메시지 추가"""
        self.message_notifications.append(new_message)

    def display_message_notifications(self):
        """모든 새 메시지 확인"""
        print("새로 온 메시지들:")

        for message in self.message_notifications:
            print(message.get_notification_message() + "\n")

```
이제 MessageNotificationManager 클래스의 add_new_message 메소드가 NotifiableMessage 클래스의 인스턴스만 파라미터로 받아야한다는 의미를 나타내는 type hinting을 한다. 그리고 display_message_notifications 메소드에서 message 인스턴스의 get_notification_message 메소드를 호출하면 된다.  이렇게 하면 MessageNotificationManager 클래스에 어떤 종류의 메시지 인스턴스가 추가되더라도 그 인스턴스는 get_notification_message 메소드를 갖고 있을테니 새로운 메시지 클래스가 생긴다고 하더라도 MessageNotificationManager 클래스는 그 코드를 수정할 필요가 없다.




